In [27]:
# 8.1 Change string resperentation of instance
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Pair(%r, %r)'%(self.x,self.y)
#         return 'Pair({0.x!r}, {0.y!r})'.format(self) format {0.x}: x attribute of arg 0
    def __str__(self):
        return '(%s,%s)'%(self.x,self.y)
#         return '({0.x!s}, {0.y!s})'.format(self)

In [28]:
pair = Pair(3,2)

In [29]:
print(pair)

(3,2)


In [30]:
pair

Pair(3, 2)

In [31]:
repr(pair)

'Pair(3, 2)'

In [43]:
8.2 
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
}
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    # Define how format method works. Format implementation is defined entirely up to the class based on code
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)


In [44]:
d = Date(2000,3,29)

In [45]:
format(d,'mdy')

'3/29/2000'

In [47]:
'The date is {:dmy}'.format(d)

'The date is 29/3/2000'

In [48]:
'The date is {:ymd}'.format(d)

'The date is 2000-3-29'

In [1]:
#8.3
from socket import socket, AF_INET, SOCK_STREAM
class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.sock = None
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None
        

In [2]:
#8.4 slots
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}/{d.day}/{d.year}',
    'dmy' : '{d.day}/{d.month}/{d.year}'
}
class Date:
    __slots__=['year','month','day'] # reduce mem footprint, cannot add new attributes to instance but only use ones in __slots__
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    # Define how format method works. Format implementation is defined entirely up to the class based on code
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [57]:
# 8.5 Encapsulation
# _method and _var means internal implementation
class A:
    def __init__(self):
        self._internal = 0 # An internal attribute
        self.public = 1 # A public attribute
    def public_method(self):
        '''
        A public method
        '''
        print("Public method used")
        self._internal_method() # better practice to run internal method
    def _internal_method(self):
        '''
        An internal method
        '''
        print("Internal method used")
        
# __private use __ for inheritance, attr cannot be overrriden if instance.__private was init diff than class.__private
B=A()
B.public_method()

Internal method used


In [50]:
# 8.6 Managed attributes
# Use property

class Person:
    def __init__(self, first_name):
        self.first_name = first_name
        self.last_name = last_name
# Getter function
    @property
    def first_name(self):
        return self._first_name
    @property
    def last_name(self):
        return self._last_name
# Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
    @last_name.setter
    def last_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
# Deleter function (optional)
    @first_name.deleter
    def first_name(self):
        raise AttributeError("Can't delete attribute")


In [4]:
# 8.7
class Amogus:
    def action(self):
        print('Amogus doing things')
class Imposter(Amogus):
    def action(self):
        print('Sabotaging')
        super().action() # Call action of parent class (superclass)

In [8]:
# 8.8
class Amogus:
    def __init__(self,name):
        self.name=name
    def action(self):
        print('Amogus doing things')
    #Getter
    @property
    def name(self):
        return self._name
    # Setter function
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value
    # Deleter function
    @name.deleter
    def name(self):
            raise AttributeError("Can't delete attribute")
class Imposter(Amogus):
    def action(self):
        print('Sabotaging')
        super().action() # Call action of parent class (superclass)
    @property
    def name(self):
        print('My name is: ')
        return super().name
    @name.setter
    def name(self, value):
        print('Setting name to: ', value)
        super(SubPerson, SubPerson).name.__set__(self, value)
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)

In [3]:
# 8.9
# Instance attr with desciptor class
class Amogus:
    def __init__(self, name):
        self.name=name
    def __get__(self):
        return self.name
    def __set__(self,value):
        if not isinstance(value, string):
            raise TypeError('Expected a string')
        self.name = value
    

In [5]:
red = Amogus("red crewmate")
red.name

'red crewmate'

In [7]:
red.name = "blue crewmate"
red.name

'blue crewmate'

In [11]:
# 8.10  lazy compute properties (compute on access), improve performance
class lazyproperty:
    def __init__(self, func):
        self.func = func
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value
import math
class Square:
    def __init__(self, width):
        self.width = width
        
    @lazyproperty #compute area when requested
    def area(self):
        print('Computing area')
        return self.width ** 2


In [16]:
# 8.11 Simplifying init of data structures
class Structure:
    _fields=[ ]
# Class variable that specifies expected fields
    
    def __init__(self, *args):
        if len(args) != len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
    # Set the arguments
    for name, value in zip(self._fields, args):
        setattr(self, name, value)
    if __name__ == '__main__':
        class Stock(Structure):
            _fields = ['name', 'shares', 'price']
        class Point(Structure):
            _fields = ['x','y']
        class Circle(Structure):
            _fields = ['radius']
            def area(self):
                return math.pi * self.radius ** 2

NameError: name 'self' is not defined

In [8]:
# 8.12 Define interface/abstract class

from abc import ABCMeta, abstractmethod # import abc module for abstract base class
class IStream(metaclass=ABCMeta):
    @abstractmethod # appear after other func definition (@keyword) if
    def read(self, maxbytes=-1):
        pass
    @abstractmethod
    def write(self, data):
        pass
# implement class:
class SocketStream(IStream):
    def read(self, maxbytes=-1):
        ...
    def write(self, data):
        ...

In [17]:
# 8.13 Desciptor
class Ten:
    def __get__(self, obj, objtype=None):
        return 10
a = Ten() # print(a) return 10

In [18]:
# 8.14 Implment custom containters, mimic common built in container
# Ex: a custom Iterable
import collections
class UselessIter(collections.abc.Iterable):
    def __iter__():# needs to be init to be valid
        return "Lol this doesnt work"

In [19]:
 a = UselessIter()

In [20]:
# 8.15. Delegating Attribute Access
# Alternate of inheritance or proxy
class A:
    def spam(self, x):
        print(f"LOLOLOL")
class B:
    def __init__(self):
        self._a = A()
    def spam(self,x):
        return self._a.spam(x)

In [1]:
# 8.16 2+ constructors in a class
import time
class Date:
# Primary constructor
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
# Alternate constructor
    @classmethod
    def today(cls):
        t = time.localtime()
        return cls(t.tm_year, t.tm_mon, t.tm_mday)

regDate = Date(2022,10,10)
todayDate = Date.today()

In [2]:
# 8.17 use __new__() to create instance without __init__()
d = Date.__new__(Date)

In [3]:
# Item is created, but no value
d

In [4]:
d.year

AttributeError: 'Date' object has no attribute 'year'

In [5]:
# Set data for d
data = {'year':2012, 'month':8, 'day':29}
for key, value in data.items():
    setattr(d, key, value)

In [21]:
# 8.18 Extend class with mixins. Useful when extending classes arent related via inheritance
class LoggedMappingMixin:
    '''
    Add logging to get/set/delete operations for debugging.
    '''
    __slots__ = ()
    def __getitem__(self, key):
        print('Getting ' + str(key))
        return super().__getitem__(key)
    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return super().__setitem__(key, value)
    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return super().__delitem__(key)
class SetOnceMappingMixin:
    '''
    Only allow a key to be set once.
    '''
    __slots__ = ()
    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(str(key) + ' already set')
        return super().__setitem__(key, value)
class StringKeysMappingMixin:
    '''
    Restrict keys to strings only
    '''
    __slots__ = ()
    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError('keys must be strings')
        return super().__setitem__(key, value)

In [22]:
# Example of implementation: mixes these classes above with mapping class
>>> class LoggedDict(LoggedMappingMixin, dict):
... pass
...
>>> d = LoggedDict()
>>> d['x'] = 23
Setting x = 23
>>> d['x']
Getting x
23
>>> del d['x']
Deleting x
>>> from collections import defaultdict
>>> class SetOnceDefaultDict(SetOnceMappingMixin, defaultdict):
... pass
...
>>> d = SetOnceDefaultDict(list)
>>> d['x'].append(2)
>>> d['y'].append(3)
>>> d['x'].append(10)
>>> d['x'] = 23
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
File "mixin.py", line 24, in __setitem__
raise KeyError(str(key) + ' already set')
KeyError: 'x already set'
>>> from collections import OrderedDict
>>> class StringOrderedDict(StringKeysMappingMixin, SetOnceMappingMixin, OrderedDict):
        pass
...
>>> d = StringOrderedDict()
>>> d['x'] = 23
>>> d[42] = 10
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
    File "mixin.py", line 45, in __setitem__
    
TypeError: keys must be strings
>>> d['x'] = 42
Traceback (most recent call last):
File "<stdin>", line 1, in <module>
File "mixin.py", line 46, in __setitem__
__slots__ = ()
File "mixin.py", line 24, in __setitem__
if key in self:
KeyError: 'x already set'
>>>

IndentationError: expected an indented block (Temp/ipykernel_23576/1378305548.py, line 3)

In [6]:
# 8.19 Stateful Objects or State Machines
class Connection:
    def __init__(self):
        self.new_state(ClosedConnectionState)
    def new_state(self, newstate):
        self._state = newstate
    # Delegate to the state class
    def read(self):
        return self._state.read(self)
    def write(self, data):
        return self._state.write(self, data)
    def open(self):
        return self._state.open(self)
    def close(self):
        return self._state.close(self)
# Connection state base class
class ConnectionState:
    @staticmethod
    def read(conn):
        raise NotImplementedError()
    @staticmethod
    def write(conn, data):
        raise NotImplementedError()
    @staticmethod
    def open(conn):
        raise NotImplementedError()
    @staticmethod
    def close(conn):
        raise NotImplementedError()
# Implementation of different states
class ClosedConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        raise RuntimeError('Not open')
    @staticmethod
    def write(conn, data):
        raise RuntimeError('Not open')
    @staticmethod
    def open(conn):
        conn.new_state(OpenConnectionState)
    @staticmethod
    def close(conn):
        raise RuntimeError('Already closed')
class OpenConnectionState(ConnectionState):
    @staticmethod
    def read(conn):
        print('reading')
    @staticmethod
    def write(conn, data):
        print('writing')
    @staticmethod
    def open(conn):
        raise RuntimeError('Already open')
    @staticmethod
    def close(conn):
        conn.new_state(ClosedConnectionState)

In [ ]:
# 8.20 call method of object given name as a string
import math
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Point({!r:},{!r:})'.format(self.x, self.y)
    def distance(self, x, y):
        return math.hypot(self.x - x, self.y - y)
p = Point(2, 3)
d = getattr(p, 'distance')(0, 0) # Calls p.distance(0, 0)

In [7]:
# 8.21 Visitor pattern
# Example: class that translate expression into operations in a stack machine
class Node:
    pass
class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand
class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right
class Add(BinaryOperator):
    pass
class Sub(BinaryOperator):
    pass
class Mul(BinaryOperator):
    pass
class Div(BinaryOperator):
    pass
class Negate(UnaryOperator):
    pass
class Number(Node):
    def __init__(self, value):
        self.value = value
class NodeVisitor: # Vistor pattern
    def visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None) 
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__))
class Evaluator(NodeVisitor): # Use visitor
    def visit_Number(self, node):
        return node.value
    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)
    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)
    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)
    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)
    def visit_Negate(self, node):
        return -node.operand

In [8]:
# Representation of 1 + 2 * (3 - 4) / 5
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)
e = Evaluator()
e.visit(t4)

0.6

In [ ]:
# 8.23 Generators+ stack for visitor pattern w/o recursion
# Replace NodeVisitor above
import types
class NodeVisitor:
    def visit(self, node):
        stack = [ node ]
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                    last_result = None
                elif isinstance(last, Node):
                    stack.append(self._visit(stack.pop()))
                else:
                    last_result = stack.pop()
            except StopIteration:
                stack.pop()
        return last_result

In [ ]:
# 8.23 cyclic data creation with less memory problems
# Ex: cyclic data struct (child node connect to parent)
# Use weakref to parent node 
import weakref
class Node:
    def __init__(self, value):
        self.value = value
        self._parent = None
        self.children = []
    def __repr__(self):
        return 'Node({!r:})'.format(self.value)
    # property that manages the parent as a weak-reference
    @property
    def parent(self):
        return self._parent if self._parent is None else self._parent()
    @parent.setter
    def parent(self, node):
        self._parent = weakref.ref(node)
    def add_child(self, child):
        self.children.append(child)
        child.parent = self

In [10]:
#8.24 Make classes support comparision operator (>=. >...)
# implement __ge__, __eq__, __le__, __gt__ or __lt__

class Item: 
    def __eq__(self, other):
        return True
class alternateItem(Item):
    def __ge__(self, other):
        return True
item1 = Item()
item2 = alternateItem()
# print(item1>=item2) Not supported
print(item1==item2) #True
print(item2>=item2) #True

True
True


In [11]:
# 8.25 Cached instance
# The class in question
class Spam:
    def __init__(self, name):
        self.name = name
# Caching support 
import weakref
_spam_cache = weakref.WeakValueDictionary()
def get_spam(name):
    if name not in _spam_cache:
        s = Spam(name)
        _spam_cache[name] = s
    else:
        s = _spam_cache[name]
    return s